# How to Transform Data Using Scikit-Learn's Pipeline

In [10]:
# Data Representation
import numpy as np
import pandas as pd

# Processing & Modeling
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

random_state = 42

## Get the data

In [6]:
link = 'https://drive.google.com/file/d/1tT0lNiDHwGQPLa3N0zSdhZcJZaa5aqF3/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+link.split('/')[-2]

In [104]:
listings_df = pd.read_csv(path)
listings_df.set_index('id', inplace=True)
listings_df.head(1)

,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07


In [105]:
listings_df.shape

(3818, 91)

In [106]:
pd.set_option('display.max_rows', 100)
# % of NaN values
missing_series = pd.Series(1 - listings_df.count() / len(listings_df))
missing_series

listing_url                         0.000000
scrape_id                           0.000000
last_scraped                        0.000000
name                                0.000000
summary                             0.046359
space                               0.149031
description                         0.000000
experiences_offered                 0.000000
neighborhood_overview               0.270299
notes                               0.420639
transit                             0.244631
thumbnail_url                       0.083814
medium_url                          0.083814
picture_url                         0.000000
xl_picture_url                      0.083814
host_id                             0.000000
host_url                            0.000000
host_name                           0.000524
host_since                          0.000524
host_location                       0.002095
host_about                          0.224987
host_response_time                  0.136983
host_respo

In [107]:
missing_series[missing_series.gt(.60)] # get the rows that contain a value greater than .60

square_feet      0.974594
monthly_price    0.602672
license          1.000000
dtype: float64

> Because the the features square_feeet, monthly_price, and license have 60% or greater missing data, we are going to drop the features for this analysis

In [108]:
listings_df = listings_df.drop(columns=['square_feet', 'monthly_price', 'license'])
listings_df.shape

(3818, 88)

In [109]:
from sklearn.model_selection import train_test_split

In [110]:
listings_df['price_float'] = listings_df['price'].replace('[\$\,]',"",regex=True).astype(float)

In [111]:
y =  listings_df.pop('price_float')
listings_df.pop('price')
X = listings_df

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [113]:
X_train_set_numerical = X_train.select_dtypes(exclude='object')
X_train_set_numerical.dtypes

scrape_id                           int64
host_id                             int64
host_listings_count               float64
host_total_listings_count         float64
latitude                          float64
longitude                         float64
accommodates                        int64
bathrooms                         float64
bedrooms                          float64
beds                              float64
guests_included                     int64
minimum_nights                      int64
maximum_nights                      int64
availability_30                     int64
availability_60                     int64
availability_90                     int64
availability_365                    int64
number_of_reviews                   int64
review_scores_rating              float64
review_scores_accuracy            float64
review_scores_cleanliness         float64
review_scores_checkin             float64
review_scores_communication       float64
review_scores_location            

In [129]:
X_train_num_subset = X_train_set_numerical[['host_listings_count', 'host_total_listings_count', 'review_scores_cleanliness']]
X_train_num_subset.head(1)

,host_listings_count,host_total_listings_count,review_scores_cleanliness
id,,,
6717555,1.0,1.0,6.0


In [130]:
X_train_num_subset.shape

(3054, 3)

In [132]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

X_train_num_subset_transformed = num_pipeline.fit_transform(X_train_num_subset)

In [133]:
X_train_num_subset_transformed

array([[-0.21361479, -0.21361479, -4.69696311],
       [-0.21361479, -0.21361479,  0.49515537],
       [-0.21361479, -0.21361479,  0.49515537],
       ...,
       [-0.14879814, -0.14879814,  0.49515537],
       [-0.21361479, -0.21361479, -4.69696311],
       [-0.18120647, -0.18120647, -3.39893349]])

In [135]:
X_train_num_subset_t_df = pd.DataFrame(X_train_num_subset_transformed, columns=X_train_num_subset.columns)
X_train_num_subset_t_df.head(1)

,host_listings_count,host_total_listings_count,review_scores_cleanliness
0,-0.213615,-0.213615,-4.696963


In [136]:
train_set_num_transformed.shape

(3054, 3)

## Review Categorical Data

In [149]:
X_train.select_dtypes(include='object').columns.tolist()

['listing_url',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'is_location_exact',
 'property_type',
 'room_type',
 'bed_type',
 'amenities',
 'weekly_price',
 'security_deposit',
 'cleaning_fee',
 'extra_people',
 'calendar_updated',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'requires_license',
 'jurisdiction_n

In [151]:
X_train.select_dtypes(include='object').head(1).T

id,6717555
listing_url,https://www.airbnb.com/rooms/6717555
last_scraped,2016-01-04
name,1 Bedroom by UW for Summer Rental
summary,"Private, furnished room in a 3-bdr apartment i..."
space,NaN
description,"Private, furnished room in a 3-bdr apartment i..."
experiences_offered,none
neighborhood_overview,NaN
notes,NaN
transit,NaN


> Review categorical data to identify features that may not make sense for prediction... for instance URLs are not necessarily useful for prediction in this context

In [152]:
boolean_columns = ['host_is_superhost', 
                    'host_has_profile_pic', 
                    'host_identity_verified', 
                    'is_location_exact',
                    'has_availability',
                    'requires_license',
                    'instant_bookable',
                    'require_guest_profile_picture',
                    'require_guest_phone_verification']

## Compose Multi-part Pipelines 

In [41]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

In [141]:
num_attribs = X_train.select_dtypes(exclude='object').columns.tolist()
cat_attribs = X_train.select_dtypes(include='object').columns.tolist()

In [143]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

train_set_prepared = full_pipeline.fit_transform(X_train)

In [144]:
train_set_prepared.shape

(3054, 49057)

## Combine Preprocessing & Prediction

In [147]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(X_train, y_train)
y_pred = full_pipeline_with_predictor.predict(X_test)

ValueError: Found unknown categories ['https://www.airbnb.com/rooms/6118359', 'https://www.airbnb.com/rooms/3726391', 'https://www.airbnb.com/rooms/3604931', 'https://www.airbnb.com/rooms/888967', 'https://www.airbnb.com/rooms/580182', 'https://www.airbnb.com/rooms/6214998', 'https://www.airbnb.com/rooms/4041868', 'https://www.airbnb.com/rooms/72743', 'https://www.airbnb.com/rooms/6360895', 'https://www.airbnb.com/rooms/7762347', 'https://www.airbnb.com/rooms/7013085', 'https://www.airbnb.com/rooms/808811', 'https://www.airbnb.com/rooms/9410043', 'https://www.airbnb.com/rooms/6748502', 'https://www.airbnb.com/rooms/6328150', 'https://www.airbnb.com/rooms/4520099', 'https://www.airbnb.com/rooms/9492036', 'https://www.airbnb.com/rooms/7075357', 'https://www.airbnb.com/rooms/108468', 'https://www.airbnb.com/rooms/7802266', 'https://www.airbnb.com/rooms/6765045', 'https://www.airbnb.com/rooms/8389809', 'https://www.airbnb.com/rooms/5978498', 'https://www.airbnb.com/rooms/8193332', 'https:/